## TAREA FINAL: Datos de vivienda

El negocio inmobiliario es uno de los motores de la economía de España, el carácter turístico de nuesta península hace que sean muchos los inversores extranjeros que se decidan por buscar una inversión de bajo riesgos en nuestras ciudades costeras o grandes urbes. Muchas de estas inversiones se hacen sobre grandes bolsas de inmuebles que deben ser analizados previamente para comprobar la rentabilidad del porfolio.

Todas las cuestiones se deben realizar sobre el conjunto de casos que representen viviendas ubicadas en zonas residenciales (alta, media y baja densidad).

Las variables de las que se compone el dataset son:

|NOMBRE VARIABLE|DESCRIPTOR|VALORES|
| --- | --- | --- |
|Order|Variable de identificación|1 a 2930|
|MS Zoning|Zona de ubicación de la vivienda|"A rural, C comercial, FV residencial flotante, I industrial, RH residencial alta densidad, RL residencial baja densidad, RM residencial media densidad"|
|Lot Frontage|Longitud de la fachada en pies||
|Lot Area|Superficie de la vivienda en pies cuadrados||
|Land Contour|Contorno del terreno circundante|"Lvl llano, Bnk Tipo bancal, HLS Ladera, Low Depresión"|
|Land Slope|Tipo de pendiente de la vivienda|" Gtl pendiente suave, Mod pendiente moderada, Sev fuerte pendiente"|
|Overall Qual|Grado de calidad de materiales y acabado de la vivienda|De 1 (Muy pobre) a 10 (Excelente)|
|Year Built|Año de construccion de la vivienda||
|Year Remod/Add|Año de última reforma de la vivienda||
|Mas Vnr Type|Tipo de revestimiento exterior|" BrkCmn Ladrillo normal, BrkFace Ladrillo visto, CBlock Bloque de cemento, None Ninguna, Stone Piedra "|
|Exter Qual|Calidad de revestimiento exterior|"Ex Excelente,Gd Bueno,TA Media,Fa Justo"|
|Bsmt Cond|Estado general del sótano|"Ex Excelente, Gd Bueno, TA Media, Fa Justo, Po Pobre,Ss sin sótano"|
|Total Bsmt SF|Superficie del sótano en pies cuadrados|
|Heating QC|Calidad de la calefacción|"Ex Excelente,Gd Bueno,TA Media,Fa Justo,Po Pobre"|
|Central Air|Aire acondicionado centralizado|"N No Y Sí"|
|Full Bath|Número de baños completo en planta||
|Half Bath|Número de aseos en planta||
|Bedroom AbvGr|Número de dormitorios en planta||
|Kitchen AbvGr|Número de cocinas en planta||
|Kitchen Qual|Calidad de cocinas|"Ex Excelente,Gd Bueno,TA Media,Fa Justo,Po Pobre"|
|TotRms AbvGrd|Número total de habitaciones excluidos los cuartos de baño||
|Garage Cars|Número de plazas de garaje||
|Garage Area|Superficie del garaje|||
|Garage Cond|Estado del garaje|"Ex Excelente,Gd Bueno,TA Media,Fa Justo,Po Pobre,Sg sin garaje"|
|Pool Area|Superficie de la piscina en pies cuadrados|
|Pool QC|Calidad de la piscina|"Ex Excelente,Gd Bueno,TA Media,Fa Justo,Sp no hay piscina"|
|Mo Sold|mes de venta||
|Yr Sold|año de venta||
|SalePrice|precio de venta en dólares||

### Inicializar y cargar el contexto spark

In [120]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
from pyspark.sql import SQLContext
from pyspark.sql import Row
import numpy as np
import pandas as pd

Se importa el archivo con los datos

In [121]:
from google.colab import files
files.upload()

Saving BDpracticafinalCSV.csv to BDpracticafinalCSV (2).csv


{'BDpracticafinalCSV.csv': b'\xef\xbb\xbfOrder;MS Zoning;Lot Frontage;Lot Area;Land Contour;Land Slope;Overall Qual;Year Built;Year Remod/Add;Mas Vnr Type;Exter Qual;Bsmt Cond;Total Bsmt SF;Heating QC;Central Air;Full Bath;Half Bath;Bedroom AbvGr;Kitchen AbvGr;Kitchen Qual;TotRms AbvGrd;Garage Cars;Garage Area;Garage Cond;Pool Area;Pool QC;Mo Sold;Yr Sold;SalePrice\r\n1;RL;141;31770;Lvl;Gtl;6;1960;1960;Stone;TA;Gd;1080;Fa;Y;1;0;3;1;TA;7;2;528;TA;0;Sp;5;2010;215000\r\n2;RH;80;11622;Lvl;Gtl;5;1961;1961;None;TA;TA;882;TA;Y;1;0;2;1;TA;5;1;730;TA;0;Sp;6;2010;105000\r\n3;RL;81;14267;Lvl;Gtl;6;1958;1958;BrkFace;TA;TA;1329;TA;Y;1;1;3;1;Gd;6;1;312;TA;0;Sp;6;2010;172000\r\n4;RL;93;11160;Lvl;Gtl;7;1968;1968;None;Gd;TA;2110;Ex;Y;2;1;3;1;Ex;8;2;522;TA;0;Sp;4;2010;244000\r\n5;RL;74;13830;Lvl;Gtl;5;1997;1998;None;TA;TA;928;Gd;Y;2;1;3;1;TA;6;2;482;TA;0;Sp;3;2010;189900\r\n6;RL;78;9978;Lvl;Gtl;6;1998;1998;BrkFace;TA;TA;926;Ex;Y;2;1;3;1;Gd;7;2;470;TA;0;Sp;6;2010;195500\r\n7;RL;41;4920;Lvl;Gtl;8;2001;200

In [122]:
raw_data = sc.textFile("BDpracticafinalCSV.csv")

Se separan los datos por ";" para crear los elementos y se elimina el encabezado.

In [123]:
raw_data = raw_data.map(lambda x: x.split(';'))
header = raw_data.first()
raw_data = raw_data.filter(lambda x: x !=header)
raw_data.take(2)

[['1',
  'RL',
  '141',
  '31770',
  'Lvl',
  'Gtl',
  '6',
  '1960',
  '1960',
  'Stone',
  'TA',
  'Gd',
  '1080',
  'Fa',
  'Y',
  '1',
  '0',
  '3',
  '1',
  'TA',
  '7',
  '2',
  '528',
  'TA',
  '0',
  'Sp',
  '5',
  '2010',
  '215000'],
 ['2',
  'RH',
  '80',
  '11622',
  'Lvl',
  'Gtl',
  '5',
  '1961',
  '1961',
  'None',
  'TA',
  'TA',
  '882',
  'TA',
  'Y',
  '1',
  '0',
  '2',
  '1',
  'TA',
  '5',
  '1',
  '730',
  'TA',
  '0',
  'Sp',
  '6',
  '2010',
  '105000']]

#### Parseado

In [124]:
raw_data.map(lambda x:x[1]).distinct().collect()

['RL', 'RH', 'C', 'Rl', 'FV', 'RM', 'I', 'A', 'rL']

In [125]:
raw_data.map(lambda x:x[9]).distinct().collect()

['None', '', 'CBlock', 'Stone', 'BrkFace', 'BrkCmn']

In [126]:
raw_data.map(lambda x:x[23]).distinct().collect()

['Fa', 'Po', '', 'TA', 'Sg', 'Gd', 'Ex']

Hay algunos problemas en los elementos referentes a la zona de ubicación de la vivienda, el tipo de revestimiento y la calidad de la calefacción.

En el primer caso, las zonas residenciales de baja intensidad (RL) están mal identificadas, counfundiendo mayúsculas y minúsculas, por lo que se homegenizan.

Para los otros dos elementos hay que completar los valores vacios, por lo que se les asigna la categoría "Otros".

Se definen las funciones para realizar el parseado y se comprueba su resultado.

In [127]:
def parseado_zona (x):                                   
    if x == 'Rl':
        xn = 'RL'
    elif x == 'rL':
        xn='RL'
    else:
        xn = x
    return (xn)

In [128]:
def parseado_blancos (x):                                          
    if x == '':
        xn = 'Otro'
    else:
        xn = x
    return (xn)

In [129]:
raw_data.map(lambda x:parseado_zona(x[1])).distinct().collect()

['RL', 'RH', 'C', 'FV', 'RM', 'I', 'A']

In [130]:
raw_data.map(lambda x:parseado_blancos(x[9])).distinct().collect()

['None', 'CBlock', 'Stone', 'BrkFace', 'Otro', 'BrkCmn']

In [131]:
raw_data.map(lambda x:parseado_blancos(x[23])).distinct().collect()

['Fa', 'Po', 'TA', 'Sg', 'Gd', 'Ex', 'Otro']

Se aplican las funciones sobre el RDD.

In [132]:
data = raw_data.map(lambda x: parseado_zona(x))
data = data.map(lambda x: parseado_blancos(x))

### ¿Cuántas viviendas distintas encontramos en el dataset? ¿Se repite alguna? Tiene sentido que haya duplicadas? ¿Qué podemos hacer con las duplicadas?

Sabiendo que existen 2936 viviviendas, se cuentan los valores únicos, es decir, no se van a contar los repetidos, por lo que viendo la diferencia se comprueba que hay 6 valores repetidos.

In [133]:
data.map(lambda x:x).count()

2936

In [134]:
data.map(lambda x:x[0]).distinct().count()

2930

En principio, no parece razonable que haya vivendas repetidas en los datos, ya que cada vivienda es un elemento único, con características propias.

Podría darse el caso de que la misma vivienda hubise sido vendida dos veces, en ese caso parace razonables que cambiase su identificación, ya que algunas características, año y precio de venta, habrían cambiado, aunque puede no ser así.

Si no se da la situación descrita arriba, habría que eliminar los duplicados.

### ¿Podrías decirme el total de inmuebles y el precio medio (Sale Price) de cada zona (MS Zoning)?

Para conocer el total de inmuebles y el precio medio por zona se comienza creando tres nuevos RDD, uno para cada zona.

También se define una función para que los valores vacios se conviertan a 0 y tipo numérico.

In [135]:
RL = data.filter(lambda x:x[1]=='RL')
RM = data.filter(lambda x:x[1]=='RM')
RH = data.filter(lambda x:x[1]=='RH')

In [136]:
def convert_to_float(x):
    if x == '':
        xn = 0
    else:
        xn = float(x)
    return(xn)  

Habiendo realizado los pasos previos, para cada zona se calcula la la media del precio(elemento 28) y se cuenta el numero de filas para conocer el total de casas.

In [137]:
print('El precio medio para la zona de baja intensidad es:',RL.map(lambda x: convert_to_float(x[28])).mean())
print('El total de viviendas en la zona de baja intensidad es:',RL.count())

El precio medio para la zona de baja intensidad es: 191131.80581241753
El total de viviendas en la zona de baja intensidad es: 2271


In [138]:
print('El precio medio para la zona de media intensidad es:',RM.map(lambda x: convert_to_float(x[28])).mean())
print('El total de viviendas en la zona de media intensidad es:',RM.count())

El precio medio para la zona de media intensidad es: 126781.39393939397
El total de viviendas en la zona de media intensidad es: 462


In [139]:
print('El precio medio para la zona de alta intensidad es:',RH.map(lambda x: convert_to_float(x[28])).mean())
print('El total de viviendas en la zona de alta intensidad es:',RH.count())

El precio medio para la zona de alta intensidad es: 136419.77777777778
El total de viviendas en la zona de alta intensidad es: 27


### Media de Total Bsmt SF por cada década de construcción calculada a partir de Year Built.

En primer lugar se determina en que año fueron construidas la primera y la última casa, para poder definir las décadas.

In [140]:
print(data.map(lambda x:x[7]).min())
print(data.map(lambda x:x[7]).max())

1872
2010


Se definen las décadas creando un nuevo RDD para cada una, filtrando por los años de cada decada y seleccionando los elementos de identificación, año, superficie del sótano y calefacción, esta última no es necesaria para resolver esta cuestión pero sí será necesaria para el ejercicio siguiente.

In [141]:
d1870s = data.filter(lambda x:'1870'<= x[7] <= '1879').map(lambda x: (x[0], x[7], x[12], x[13]))
d1880s = data.filter(lambda x:'1880'<= x[7] <= '1889').map(lambda x: (x[0], x[7], x[12], x[13]))
d1890s = data.filter(lambda x:'1890'<= x[7] <= '1899').map(lambda x: (x[0], x[7], x[12], x[13]))
d1900s = data.filter(lambda x:'1900'<= x[7] <= '1909').map(lambda x: (x[0], x[7], x[12], x[13]))
d1910s = data.filter(lambda x:'1910'<= x[7] <= '1919').map(lambda x: (x[0], x[7], x[12], x[13]))
d1920s = data.filter(lambda x:'1920'<= x[7] <= '1929').map(lambda x: (x[0], x[7], x[12], x[13]))
d1930s = data.filter(lambda x:'1930'<= x[7] <= '1939').map(lambda x: (x[0], x[7], x[12], x[13]))
d1940s = data.filter(lambda x:'1940'<= x[7] <= '1949').map(lambda x: (x[0], x[7], x[12], x[13]))
d1950s = data.filter(lambda x:'1950'<= x[7] <= '1959').map(lambda x: (x[0], x[7], x[12], x[13]))
d1960s = data.filter(lambda x:'1960'<= x[7] <= '1969').map(lambda x: (x[0], x[7], x[12], x[13]))
d1970s = data.filter(lambda x:'1970'<= x[7] <= '1979').map(lambda x: (x[0], x[7], x[12], x[13]))
d1980s = data.filter(lambda x:'1980'<= x[7] <= '1989').map(lambda x: (x[0], x[7], x[12], x[13]))
d1990s = data.filter(lambda x:'1990'<= x[7] <= '1999').map(lambda x: (x[0], x[7], x[12], x[13]))
d2000s = data.filter(lambda x:'2000'<= x[7] <= '2009').map(lambda x: (x[0], x[7], x[12], x[13]))
d2010s = data.filter(lambda x:'2010'<= x[7] <= '2019').map(lambda x: (x[0], x[7], x[12], x[13]))

Sobre los nuevos RDDs se, en el segundo elemento, referida a la superficie del sótano se aplica la función para convertir a formato 'int' y se calcula la media de la misma. Se imprimen los resultados.

In [142]:
print('Superficie media sotano 1870s:',d1870s.map(lambda x: convert_to_float(x[2])).mean())
print('Superficie media sotano 1880s:',d1880s.map(lambda x: convert_to_float(x[2])).mean())  
print('Superficie media sotano 1890s:',d1890s.map(lambda x: convert_to_float(x[2])).mean())
print('Superficie media sotano 1900s:',d1900s.map(lambda x: convert_to_float(x[2])).mean())
print('Superficie media sotano 1910s:',d1910s.map(lambda x: convert_to_float(x[2])).mean())
print('Superficie media sotano 1920s:',d1920s.map(lambda x: convert_to_float(x[2])).mean())
print('Superficie media sotano 1930s:',d1930s.map(lambda x: convert_to_float(x[2])).mean())
print('Superficie media sotano 1940s:',d1940s.map(lambda x: convert_to_float(x[2])).mean())
print('Superficie media sotano 1950s:',d1950s.map(lambda x: convert_to_float(x[2])).mean())
print('Superficie media sotano 1960s:',d1960s.map(lambda x: convert_to_float(x[2])).mean())
print('Superficie media sotano 1970s:',d1970s.map(lambda x: convert_to_float(x[2])).mean())
print('Superficie media sotano 1980s:',d1980s.map(lambda x: convert_to_float(x[2])).mean())
print('Superficie media sotano 1990s:',d1990s.map(lambda x: convert_to_float(x[2])).mean())
print('Superficie media sotano 2000s:',d2000s.map(lambda x: convert_to_float(x[2])).mean())
print('Superficie media sotano 2010s:',d2010s.map(lambda x: convert_to_float(x[2])).mean())

Superficie media sotano 1870s: 761.0
Superficie media sotano 1880s: 836.0
Superficie media sotano 1890s: 834.2
Superficie media sotano 1900s: 710.55
Superficie media sotano 1910s: 778.5272727272727
Superficie media sotano 1920s: 828.1428571428571
Superficie media sotano 1930s: 771.6697247706422
Superficie media sotano 1940s: 700.7417218543046
Superficie media sotano 1950s: 960.7411764705882
Superficie media sotano 1960s: 1082.6722689075627
Superficie media sotano 1970s: 953.9178082191784
Superficie media sotano 1980s: 1086.0000000000002
Superficie media sotano 1990s: 1124.0863095238103
Superficie media sotano 2000s: 1311.3997445721573
Superficie media sotano 2010s: 1573.3333333333333


### ¿Cuál es la decada de construcción con viviendas mejor acondicionadas para el frío (Heating QC)?

Desdel los RDDs anteriores para cada década, se filtran aquellas obervaciones con una calidad de la calefacción excelente (Ex) o buena (Gd), creando un nuevo RDD con los resultados.

In [143]:
heat1870s = d1870s.filter(lambda x: x[3]=='Ex' and 'Gd').map(lambda x: (x[0], x[3]))
heat1880s = d1880s.filter(lambda x: x[3]=='Ex' and 'Gd').map(lambda x: (x[0], x[3]))
heat1890s = d1890s.filter(lambda x: x[3]=='Ex' and 'Gd').map(lambda x: (x[0], x[3]))
heat1900s = d1900s.filter(lambda x: x[3]=='Ex' and 'Gd').map(lambda x: (x[0], x[3]))
heat1910s = d1910s.filter(lambda x: x[3]=='Ex' and 'Gd').map(lambda x: (x[0], x[3]))
heat1920s = d1920s.filter(lambda x: x[3]=='Ex' and 'Gd').map(lambda x: (x[0], x[3]))
heat1930s = d1930s.filter(lambda x: x[3]=='Ex' and 'Gd').map(lambda x: (x[0], x[3]))
heat1940s = d1940s.filter(lambda x: x[3]=='Ex' and 'Gd').map(lambda x: (x[0], x[3]))
heat1950s = d1950s.filter(lambda x: x[3]=='Ex' and 'Gd').map(lambda x: (x[0], x[3]))
heat1960s = d1960s.filter(lambda x: x[3]=='Ex' and 'Gd').map(lambda x: (x[0], x[3]))
heat1970s = d1970s.filter(lambda x: x[3]=='Ex' and 'Gd').map(lambda x: (x[0], x[3]))
heat1980s = d1980s.filter(lambda x: x[3]=='Ex' and 'Gd').map(lambda x: (x[0], x[3]))
heat1990s = d1990s.filter(lambda x: x[3]=='Ex' and 'Gd').map(lambda x: (x[0], x[3]))
heat2000s = d2000s.filter(lambda x: x[3]=='Ex' and 'Gd').map(lambda x: (x[0], x[3]))
heat2010s = d2010s.filter(lambda x: x[3]=='Ex' and 'Gd').map(lambda x: (x[0], x[3]))

Con los nuevos RDDs se cuentan las filas para ver en que década hay más casas acondicionadas de manera buena o excelente.

In [144]:
print('Casas con calefacciones excelentes en 1870s:',heat1870s.count())
print('Casas con calefacciones excelentes en 1880s:',heat1880s.count())
print('Casas con calefacciones excelentes en 1890s:',heat1890s.count())
print('Casas con calefacciones excelentes en 1900s:',heat1900s.count())
print('Casas con calefacciones excelentes en 1910s:',heat1910s.count())
print('Casas con calefacciones excelentes en 1920s:',heat1920s.count())
print('Casas con calefacciones excelentes en 1930s:',heat1930s.count())
print('Casas con calefacciones excelentes en 1940s:',heat1940s.count())
print('Casas con calefacciones excelentes en 1950s:',heat1950s.count())
print('Casas con calefacciones excelentes en 1960s:',heat1960s.count())
print('Casas con calefacciones excelentes en 1970s:',heat1970s.count())
print('Casas con calefacciones excelentes en 1980s:',heat1980s.count())
print('Casas con calefacciones excelentes en 1990s:',heat1990s.count())
print('Casas con calefacciones excelentes en 2000s:',heat2000s.count())
print('Casas con calefacciones excelentes en 2010s:',heat2000s.count())

Casas con calefacciones excelentes en 1870s: 0
Casas con calefacciones excelentes en 1880s: 2
Casas con calefacciones excelentes en 1890s: 5
Casas con calefacciones excelentes en 1900s: 13
Casas con calefacciones excelentes en 1910s: 40
Casas con calefacciones excelentes en 1920s: 52
Casas con calefacciones excelentes en 1930s: 41
Casas con calefacciones excelentes en 1940s: 56
Casas con calefacciones excelentes en 1950s: 98
Casas con calefacciones excelentes en 1960s: 88
Casas con calefacciones excelentes en 1970s: 68
Casas con calefacciones excelentes en 1980s: 28
Casas con calefacciones excelentes en 1990s: 251
Casas con calefacciones excelentes en 2000s: 755
Casas con calefacciones excelentes en 2010s: 755


### ¿Cuáles son las 10 viviendas que se vendieron por un precio más elevado por metro cuadrado en el año 2009?

Se seleccionan las viviendas vendidas en 2009 mediante un filtro y se divide para cada una el precio de venta entre la superficie.

Para ver las 10 de mayor precio se ordena en orden descendente el precio por superficie.

In [145]:
ventas2009 = data.filter(lambda x: x[27]=='2009').map(lambda x: (x[0],(convert_to_float(x[28])/convert_to_float(x[3])))).reduceByKey(lambda a,b: a+a)
top10 = ventas2009.sortBy(lambda x: convert_to_float(x[1]), ascending=False)
top10.take(10)

[('566', 97.48263888888889),
 ('936', 95.38461538461539),
 ('934', 92.06989247311827),
 ('464', 76.27865961199295),
 ('935', 75.06203473945409),
 ('408', 71.13095238095238),
 ('407', 70.23809523809524),
 ('405', 66.66666666666667),
 ('933', 66.35891517599538),
 ('403', 66.07142857142857)]

### Media anual por zonas del precio de venta y metros cuadrados.

Se selecciona el año, el precio y la superficie.

Mediante reduceByKey se suman los precios y superficies y se cra un contador, en el lambda posterior se realiza la media, dividiendo las sumas entre el contador.

In [146]:
pmRL = RL.map(lambda x: (x[7], [convert_to_float(x[28]), convert_to_float(x[3]), 1]))
pmRL = pmRL.reduceByKey(lambda x,y: [x[0]+y[0], x[1]+y[1], x[2]+y[2]]).map(lambda x: (x[0], x[1][0]/x[1][2], x[1][1]/x[1][2]))
pmRL = pmRL.sortBy(lambda x: convert_to_float(x[0]), ascending=False)
pmRL.take(10)

[('2010', 290716.0, 9377.0),
 ('2009', 308183.73333333334, 11567.066666666668),
 ('2008', 331959.25641025644, 13712.384615384615),
 ('2007', 271814.7, 10846.633333333333),
 ('2006', 265620.2831858407, 11017.079646017699),
 ('2005', 250779.89915966385, 10319.81512605042),
 ('2004', 231581.484375, 10847.75),
 ('2003', 247085.01408450704, 10226.507042253521),
 ('2002', 226886.66666666666, 10523.047619047618),
 ('2001', 241720.6896551724, 10589.344827586207)]

In [147]:
pmRM = RM.map(lambda x: (x[7], [convert_to_float(x[28]), convert_to_float(x[3]), 1]))
pmRM = pmRM.reduceByKey(lambda x,y: [x[0]+y[0], x[1]+y[1], x[2]+y[2]]).map(lambda x: (x[0], x[1][0]/x[1][2], x[1][1]/x[1][2]))
pmRM = pmRM.sortBy(lambda x: convert_to_float(x[0]), ascending=False)
pmRM.take(10)

[('2008', 267750.0, 7711.5),
 ('2007', 262232.0, 3843.0),
 ('2006', 305000.0, 5159.0),
 ('2005', 176571.42857142858, 3971.1428571428573),
 ('2004', 179546.5, 3902.7),
 ('2003', 146093.75, 5822.75),
 ('1999', 165000.0, 4500.0),
 ('1998', 202375.0, 5815.75),
 ('1997', 195200.0, 7529.2),
 ('1996', 140000.0, 6710.0)]

In [148]:
pmRH = RH.map(lambda x: (x[7], [convert_to_float(x[28]), convert_to_float(x[3]), 1]))
pmRH = pmRH.reduceByKey(lambda x,y: [x[0]+y[0], x[1]+y[1], x[2]+y[2]]).map(lambda x: (x[0], x[1][0]/x[1][2], x[1][1]/x[1][2]))
pmRH = pmRH.sortBy(lambda x: convert_to_float(x[0]), ascending=False)
pmRH.take(10)

[('2002', 190000.0, 8773.0),
 ('2001', 155000.0, 4113.0),
 ('1999', 184000.0, 5858.0),
 ('1998', 157000.0, 4059.0),
 ('1997', 177000.0, 10943.0),
 ('1993', 137000.0, 3612.0),
 ('1977', 144000.0, 8400.0),
 ('1967', 160000.0, 7200.0),
 ('1966', 107000.0, 8900.0),
 ('1965', 119900.0, 7800.0)]

### ¿Podrías decirme el total de recaudación de las casas de revistimiento (Mas Vnr Type) de piedra con respecto a las de ladrillo? ¿Hay diferencia significativa?

Se filtran por un lado aquellos datos con el revestimiento de ladrillo y por otro los de revestimiento de piedra, creando un RDD para cada uno con los elementos de identificación y precio.

In [149]:
Br = data.filter(lambda x : x[9]=='BrkCmn').map(lambda x:(x[0],x[28]))
St = data.filter(lambda x : x[9]=='Stone').map(lambda x:(x[0],x[28]))

In [ ]:
Para cada uno de los RDDs sumamos el precio de todas las viviviendas.

In [150]:
incBr = Br.map(lambda x: convert_to_float(x[1])).sum()
print('Total de ingresos vicendas con revestimiento de ladrillo:',incBr)

Total de ingresos vicendas con revestimiento de ladrillo: 3504975.0


In [151]:
incSt = St.map(lambda x: convert_to_float(x[1])).sum()
print('Total de ingresos vicendas con revestimiento de piedra:',incSt)

Total de ingresos vicendas con revestimiento de piedra: 65196277.0


Se aprecia que las viviendas con revestimiento de prieda han tenido unos ingresos mucho más elevados, 65 millones frente a 3.

### ¿Cuánto son más caras las viviendas con 2 cocinas, con 2 o más plazas de garaje que las que tienen 1 cocina y 1 plaza de garaje? Comparar medias y cuartiles de ambos casos

Se crean dos RDDs filtrando aquellas viviendas con dos cocinas y dos o más garajes y aquellas que tienen una y uno.

In [152]:
dos = data.filter(lambda x: x[18] == '2' and x[21] > '2').map(lambda x: (x[0], x[28]))
uno = data.filter(lambda x: x[18] == '1' and x[21] == '1').map(lambda x: (x[0], x[28]))

Para cada RDD se calcula la media del precio.

In [153]:
print('Media precio viviendas con dos cocinas y dos o más garajes:',dos.map(lambda x: convert_to_float(x[1])).mean())
print('Media precio viviendas con una cocina y un garaje:',uno.map(lambda x: convert_to_float(x[1])).mean())

Media precio viviendas con dos cocinas y dos o más garajes: 180611.11111111112
Media precio viviendas con una cocina y un garaje: 127367.04161248378
